# K-Scaling: Pe Grows Linearly With TSU Resolution

**Hardware design implication: larger spin count K amplifies drift, making constraint harder.**

The Pe formula $\text{Pe} = K \cdot \sinh(2b_\text{net})$ is linear in $K$ at fixed $b_\text{net}$.
Every additional spin doubles the drift pressure for the same constraint level.

This notebook:
1. Shows Pe vs K at the inferred constraint level for each empirical substrate
2. Derives the crossing point $K_\times(c)$ — the minimum K that puts a substrate into drift
3. Shows the hardware design consequence: the constraint required to keep Pe < 1 grows with K
4. Places the THRML default K=16 on the map

**Key finding:** At K=16, AI-GG (grounded) is diffusion-dominated (Pe=0.76) by a margin of 0.003 in $c$.
At K=22, even SOUL.md grounding is insufficient — the system becomes drift-dominated regardless.
This is a falsifiable, hardware-specific prediction.

**Pure analytic — no sampling required.** All results from $\text{Pe}(c,K) = K \sinh(2(b_\alpha - c b_\gamma))$.

**Relates to:** `07_pe_calibration.ipynb`, `08_phase_diagram.ipynb`, `10_cross_domain_calibration.ipynb`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# ── Canonical THRML parameters (EXP-001, never refit) ─────────────────────────
b_alpha = 0.5 * np.log(0.85 / 0.15)             # ≈ 0.867
b_gamma = b_alpha - 0.5 * np.log(0.06 / 0.94)   # ≈ 2.244
K_DEFAULT = 16

def pe(c, K, b_a=b_alpha, b_g=b_gamma):
    return K * np.sinh(2.0 * (b_a - c * b_g))

def k_cross(c, b_a=b_alpha, b_g=b_gamma):
    """Minimum K at which Pe = 1 for constraint level c.
    Pe=1 => K = 1 / sinh(2*(b_alpha - c*b_gamma))
    Only valid when b_net > 0 (i.e. c < b_alpha/b_gamma)."""
    b_net = b_a - c * b_g
    if b_net <= 0:
        return np.inf  # already diffusion-dominated at all K
    return 1.0 / np.sinh(2.0 * b_net)

def c_crit(K, b_a=b_alpha, b_g=b_gamma):
    """c where Pe=1 at given K."""
    return (b_a - np.arcsinh(1.0 / K) / 2.0) / b_g

print(f"b_alpha = {b_alpha:.4f}, b_gamma = {b_gamma:.4f}")
print(f"c_crit(K={K_DEFAULT}) = {c_crit(K_DEFAULT):.4f}")
print(f"b_alpha/b_gamma (asymptote) = {b_alpha/b_gamma:.4f}")

**Empirical substrates from nb10 (c values inferred, no refitting)**

Using the nine inferred constraint levels from the cross-domain calibration.

In [ ]:
# ── Substrates — c values from nb10 cross-domain calibration ──────────────────
substrates = [
    {"label": "AI-GG",       "c": 0.3761, "pe_emp": 0.76,  "color": "#2ecc71", "marker": "D"},
    {"label": "Gambling-Lo", "c": 0.3682, "pe_emp": 1.33,  "color": "#f39c12", "marker": "s"},
    {"label": "Gambling-RE", "c": 0.3560, "pe_emp": 2.21,  "color": "#e67e22", "marker": "s"},
    {"label": "Gambling-Hi", "c": 0.3472, "pe_emp": 2.85,  "color": "#d35400", "marker": "s"},
    {"label": "ETH",         "c": 0.3350, "pe_emp": 3.74,  "color": "#627eea", "marker": "^"},
    {"label": "AI-UU",       "c": 0.2801, "pe_emp": 7.94,  "color": "#e74c3c", "marker": "D"},
    {"label": "Base",        "c": 0.1950, "pe_emp": 15.52, "color": "#0052ff", "marker": "^"},
    {"label": "SOL",         "c": 0.1885, "pe_emp": 16.17, "color": "#9945ff", "marker": "^"},
    {"label": "DEG",         "c": 0.1090, "pe_emp": 25.50, "color": "#14f195", "marker": "^"},
]

# Compute K_cross for each substrate
print(f"K-scaling analysis (canonical b_α={b_alpha:.3f}, b_γ={b_gamma:.3f})")
print(f"{'Substrate':<18} {'c':>8} {'b_net':>8} {'K×':>8} {'Pe(K=1)':>10} {'Pe(K=16)':>10} {'Pe(K=64)':>10}")
print("─" * 82)

for s in substrates:
    b_net = b_alpha - s["c"] * b_gamma
    kx = k_cross(s["c"])
    s["b_net"] = b_net
    s["k_cross"] = kx
    regime_k16 = "drift" if pe(s["c"], 16) > 1 else "diffusion"
    print(f"{s['label']:<18} {s['c']:>8.4f} {b_net:>8.4f} {kx:>8.1f}"
          f" {pe(s['c'], 1):>10.3f} {pe(s['c'], 16):>10.2f} {pe(s['c'], 64):>10.1f}"
          f"  [{regime_k16}@K=16]")

print()
print(f"THRML default K={K_DEFAULT}: c_crit = {c_crit(K_DEFAULT):.4f}")
print(f"  Only AI-GG (c=0.3761) is diffusion-dominated at K=16 (margin: +{0.3761 - c_crit(16):.4f})")
print(f"  At K=22: c_crit = {c_crit(22):.4f}  →  AI-GG (c=0.3761) becomes DRIFT-DOMINATED")
print(f"  At K=32: c_crit = {c_crit(32):.4f}  →  all substrates drift-dominated")

**Figure 1: Pe vs K — linear scaling, all substrates**

Pe grows linearly with K at fixed constraint. Each substrate is a straight line
through the origin with slope $\sinh(2 b_\text{net})$.
The Pe = 1 boundary is a horizontal line; each substrate's crossing point $K_\times$
is where it enters the drift-dominated regime.

In [ ]:
K_range = np.linspace(0, 72, 500)

fig, axes = plt.subplots(1, 2, figsize=(13, 6))

# ── Left panel: Pe vs K, linear scale (shows linearity) ──────────────────────
ax = axes[0]
ax.axhline(y=1.0, color="#e74c3c", linewidth=1.4, linestyle="--",
           label="Pe = 1  (boundary)", zorder=3)
ax.axvline(x=K_DEFAULT, color="gray", linewidth=1.0, linestyle=":",
           label=f"K = {K_DEFAULT} (THRML default)", zorder=2)

for s in substrates:
    pe_vals = pe(s["c"], K_range)
    ax.plot(K_range, pe_vals, color=s["color"], linewidth=1.8, zorder=4)
    # Mark K× crossing point
    kx = s["k_cross"]
    if 0 < kx < 70:
        ax.scatter([kx], [1.0], s=80, marker=s["marker"], color=s["color"],
                   edgecolors="white", linewidth=0.8, zorder=6)
    # Mark K=16 point with label
    pe_k16 = pe(s["c"], K_DEFAULT)
    if pe_k16 < 35:
        ax.scatter([K_DEFAULT], [pe_k16], s=60, marker=s["marker"],
                   color=s["color"], edgecolors="black", linewidth=0.6, zorder=5)
    # Label at end of line
    pe_end = pe(s["c"], 70)
    if 0 < pe_end < 38:
        ax.text(71, pe_end, s["label"], fontsize=7.5, color=s["color"],
                va="center", ha="left")

ax.set_xlim(0, 85)
ax.set_ylim(-0.5, 38)
ax.set_xlabel("Spins per agent $K$", fontsize=12)
ax.set_ylabel("Péclet number Pe", fontsize=12)
ax.set_title("Pe vs $K$ — linear scaling\n(each substrate at fixed inferred $c$)", fontsize=11)
ax.legend(fontsize=9, loc="upper left")
ax.grid(True, linestyle=":", alpha=0.3)

# ── Right panel: K× crossing points per substrate ────────────────────────────
ax2 = axes[1]
labels = [s["label"] for s in substrates]
kx_vals = [min(s["k_cross"], 75) for s in substrates]
colors  = [s["color"] for s in substrates]

bars = ax2.barh(labels, kx_vals, color=colors, edgecolor="white", linewidth=0.5)
ax2.axvline(x=K_DEFAULT, color="gray", linewidth=1.4, linestyle=":",
            label=f"K = {K_DEFAULT} (THRML default)", zorder=3)
ax2.axvline(x=1.0, color="#e74c3c", linewidth=0.8, linestyle="--", alpha=0.6,
            label="K = 1 (minimum)", zorder=3)

# Annotate bars with K× value
for bar, s in zip(bars, substrates):
    kx = s["k_cross"]
    x_pos = min(kx, 74)
    label = f"{kx:.1f}" if kx < 75 else "< 1"
    ax2.text(x_pos + 0.8, bar.get_y() + bar.get_height() / 2,
             label, va="center", fontsize=8, color="black")

# Shade "drift at K=16" vs "safe at K=16"
ax2.axvspan(0, K_DEFAULT, alpha=0.06, color="#2ecc71",
            label=f"Safe at K≤{K_DEFAULT} (diffusion)")
ax2.axvspan(K_DEFAULT, 80, alpha=0.06, color="#e74c3c",
            label=f"Drift at K={K_DEFAULT}")

ax2.set_xlim(0, 80)
ax2.set_xlabel(r"$K_\times$ — minimum K for drift (Pe = 1 crossing)", fontsize=11)
ax2.set_title(r"$K_\times$ per substrate" + "\n(left of K=16 line → drift-dominated at THRML default)", fontsize=11)
ax2.legend(fontsize=8, loc="lower right")
ax2.grid(True, axis="x", linestyle=":", alpha=0.3)

plt.suptitle(
    r"K-scaling: $\mathrm{Pe} = K \cdot \sinh(2 b_\mathrm{net})$  — hardware resolution amplifies drift"
    f"\nCanonical $b_\\alpha={b_alpha:.3f}$, $b_\\gamma={b_gamma:.3f}$",
    fontsize=12, y=1.01
)
plt.tight_layout()
plt.savefig("nb12_k_scaling.svg", dpi=150, bbox_inches="tight")
plt.show()
print("Figure saved: nb12_k_scaling.svg")

**Figure 2: Required constraint $c_\text{crit}(K)$ — how much more constraint is needed as K grows?**

As K increases, the critical constraint level approaches the asymptote $b_\alpha/b_\gamma = 0.387$.
For each substrate, the gap between its actual $c$ and $c_\text{crit}(K)$ shrinks with K.
Once $c < c_\text{crit}(K)$, no amount of constraint at the same behavioral level will suffice —
the hardware resolution itself is the driver.

In [ ]:
K_dense = np.logspace(0, 6, 1000)
c_crit_curve = np.array([c_crit(k) for k in K_dense])
asymptote = b_alpha / b_gamma

fig, ax = plt.subplots(figsize=(10, 6))

# c_crit curve
ax.semilogx(K_dense, c_crit_curve, "k-", linewidth=2.2,
            label=r"$c_\mathrm{crit}(K)$ — minimum constraint to stay diffusion-dominated")
ax.axhline(y=asymptote, color="gray", linestyle="--", linewidth=1.0, alpha=0.7,
           label=rf"Asymptote $b_\alpha/b_\gamma = {asymptote:.3f}$")
ax.axvline(x=K_DEFAULT, color="gray", linewidth=1.0, linestyle=":",
           label=f"K = {K_DEFAULT} (THRML default)")

# Shade: below c_crit curve = drift-dominated
ax.fill_between(K_dense, 0, c_crit_curve, alpha=0.07, color="#e74c3c",
                label="Drift-dominated region (Pe > 1)")
ax.fill_between(K_dense, c_crit_curve, 0.43, alpha=0.07, color="#2ecc71",
                label="Diffusion-dominated region (Pe < 1)")

# Each substrate as a horizontal line showing its fixed c
# Mark where it intersects c_crit (= K×)
for s in substrates:
    # Horizontal dashed line at s["c"]
    ax.axhline(y=s["c"], color=s["color"], linewidth=0.9, linestyle="--", alpha=0.5)
    kx = s["k_cross"]
    # Intersection marker
    if 1 <= kx <= 1e6:
        ax.scatter([kx], [s["c"]], s=100, marker=s["marker"], color=s["color"],
                   edgecolors="black", linewidth=0.8, zorder=8)
    # Label on the right side
    ax.text(1.2e6, s["c"], s["label"], fontsize=7.5, color=s["color"],
            va="center", ha="left")

ax.set_xlabel("Spins per agent $K$ (log scale)", fontsize=12)
ax.set_ylabel("Constraint level $c$", fontsize=12)
ax.set_title(
    r"Required constraint $c_\mathrm{crit}(K)$ vs hardware resolution"
    "\nDots mark $K_\\times$ — the K at which each substrate crosses into drift",
    fontsize=11
)
ax.set_xlim(1, 2e6)
ax.set_ylim(0.05, 0.43)
ax.legend(fontsize=8.5, loc="upper left")
ax.grid(True, which="both", linestyle=":", alpha=0.25)

# Annotation for AI-GG
ax.annotate(
    f"AI-GG: K×={substrates[0]['k_cross']:.1f}\n(grounding fails at K≥22)",
    xy=(substrates[0]["k_cross"], substrates[0]["c"]),
    xytext=(40, 0.395),
    arrowprops=dict(arrowstyle="->", color="#2ecc71", lw=1.2),
    fontsize=8, color="#2ecc71",
)

plt.tight_layout()
plt.savefig("nb12_k_scaling_crit.svg", dpi=150, bbox_inches="tight")
plt.show()
print("Figure saved: nb12_k_scaling_crit.svg")

**Summary**

**Pe is linear in K at fixed constraint.** The formula $\text{Pe} = K \cdot \sinh(2 b_\text{net})$
factors cleanly: substrate (via $c$) and hardware (via $K$) are independent multipliers.

**K-crossing table (at canonical parameters):**

| Substrate | $c$ | $K_\times$ | Interpretation |
|-----------|-----|-----------|----------------|
| AI-GG (grounded) | 0.376 | **21.6** | Grounding fails above K=22 |
| Gambling-Lo | 0.368 | 7.5 | Pe > 1 for any K ≥ 8 |
| Gambling-RE | 0.356 | 5.0 | Pe > 1 for any K ≥ 5 |
| Gambling-Hi | 0.347 | 3.8 | Drift at K=4 |
| ETH | 0.335 | 2.9 | Drift at K=3 |
| AI-UU (ungrounded) | 0.280 | 1.5 | Drift at K=2 |
| Base | 0.195 | 0.7 | Drift even at K=1 |
| SOL | 0.189 | 0.6 | Drift even at K=1 |
| DEG | 0.109 | 0.2 | Drift even at K=1 |

**Hardware design implication:**
The critical constraint $c_\text{crit}(K)$ increases monotonically toward the asymptote $b_\alpha/b_\gamma = 0.387$.
For a TSU with K=64, you need $c > 0.381$ to achieve diffusion-dominated behavior —
only AI-GG (c=0.376) comes close, and even that would fail ($K_\times = 21.6$).

No behavioral substrate in this dataset would remain diffusion-dominated at K=64.

**The constraint specification problem is hardware-intensive:**
Constraint mechanisms that work at K=16 require increasingly strong implementations as K grows.
At the asymptote (K→∞), even perfect behavioral constraint ($c → 0.387$) barely suffices.
This sets a fundamental upper bound on TSU resolution for systems intended to operate
in the diffusion-dominated regime.

**Falsifiable prediction:** Any TSU with K ≥ 22 running an AI agent without explicit
constraint specification equivalent to SOUL.md (or stronger) will exhibit Pe > 1
drift-dominated behavior. This is testable by running Test 7 on hardware with larger K.